In [3]:
# ==============================
# 1. Imports
# ==============================
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os

print("Train classes:", os.listdir("../data/train"))
print("Validation classes:", os.listdir("../data/val"))

# ==============================
# 2. Configuration
# ==============================
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
FINE_TUNE_EPOCHS = 5
LEARNING_RATE = 0.0001

TRAIN_DIR = "../data/train"
VAL_DIR = "../data/val"
MODEL_PATH = "../models/model.h5"
LABELS_PATH = "../models/labels.txt"

# ==============================
# 3. Data Generators
# ==============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

NUM_CLASSES = train_generator.num_classes
CLASS_LABELS = train_generator.class_indices

# ==============================
# 4. Save Labels
# ==============================
with open(LABELS_PATH, "w") as f:
    for label in CLASS_LABELS:
        f.write(f"{label}\n")

print("Class labels saved.")

# ==============================
# 5. Load Pretrained VGG16
# ==============================
base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

# ==============================
# 6. Custom Classification Head
# ==============================
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# ==============================
# 7. Compile Model
# ==============================
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ==============================
# 8. Initial Training
# ==============================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# ==============================
# 9. Fine-Tuning (Unfreeze Top Layers)
# ==============================
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=FINE_TUNE_EPOCHS
)

# ==============================
# 10. Save Model
# ==============================
model.save(MODEL_PATH)
print("Model saved successfully.")


Train classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Validation classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Found 1766 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Class labels saved.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,139,014 (80.64 MB)

 Trainable params: 6,424,326 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 647s 11s/step - accuracy: 0.4134 - loss: 1.4833 - val_accuracy: 0.5676 - val_loss: 1.1233
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 390s 7s/step - accuracy: 0.5566 - loss: 1.1569 - val_accuracy: 0.6658 - val_loss: 0.9918
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 364s 7s/step - accuracy: 0.6291 - loss: 0.9990 - val_accuracy: 0.6737 - val_loss: 0.8694
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 372s 7s/step - accuracy: 0.6676 - loss: 0.8971 - val_accuracy: 0.7056 - val_loss: 0.8072
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 421s 8s/step - accuracy: 0.6886 - loss: 0.8360 - val_accuracy: 0.7401 - val_loss: 0.7661
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 375s 7s/step - accuracy: 0.7163 - loss: 0.7818 - val_accuracy: 0.7401 - val_loss: 0.7491
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step - accuracy: 0.7305 - loss: 0.7443 - val_accuracy: 0.7401 - val_loss: 0.6993
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 415s 7s/step - accuracy: 0.7486 - loss: 0.6786 - val_accuracy: 0.7639 - 

Model saved successfully.
